In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import sys
sys.path.append('../code')
import features
import dataframe_utilities as util

# Load Data

In [19]:
games = features.get_games()
batting = features.get_batting()
pitching = features.get_pitching()
pitchers = features.get_pitchers()

# Features

In [20]:
df = features.get_game_df()


In [21]:
df['home_pitcher'] = df['home_pitcher'].fillna('No Pitcher')
df['away_pitcher'] = df['away_pitcher'].fillna('No Pitcher')

In [22]:
df

,game_id,home_team_abbr,away_team_abbr,date,is_night_game,home_team_win,home_pitcher,away_pitcher
0,NYMSTL20240301D,STL,NYM,2024-03-01,True,True,No Pitcher,No Pitcher
1,SDMIL20240301D,MIL,SD,2024-03-01,True,True,No Pitcher,No Pitcher
2,TORNYY20240301N,NYY,TOR,2024-03-01,True,True,No Pitcher,No Pitcher
3,PHIDET20240301D,DET,PHI,2024-03-01,True,True,No Pitcher,No Pitcher
4,TEXSF20240301D,SF,TEX,2024-03-01,True,True,No Pitcher,No Pitcher
...,...,...,...,...,...,...,...,...
930,SFCOL20240508N,COL,SF,2024-05-08,True,False,No Pitcher,No Pitcher
931,ATLCWS20240627D,CWS,ATL,2024-06-27,True,False,No Pitcher,No Pitcher
932,CLEMIN20240809D,MIN,CLE,2024-08-09,True,False,No Pitcher,No Pitcher
933,MILCIN20240830D,CIN,MIL,2024-08-30,True,False,No Pitcher,No Pitcher


## Add Small Feats

In [23]:
df = features.add_trueskill_ratings(df)
df = features.add_rest_durations(df)
df.shape

(935, 17)

In [24]:
# Assuming df['date'] is a column with date information in string format
date = pd.to_datetime(df['date'])
df['season'] = date.dt.year
df['month'] = date.dt.month
# Use .isocalendar().week for ISO week number
df['week_num'] = date.dt.isocalendar().week
df['dow'] = date.dt.weekday.astype(int)

In [25]:
df['dh_game_no'] = pd.to_numeric(df['game_id'].str[-1:],errors='coerce')
df['date'] = (pd.to_datetime(df['date']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s') #epoch time

In [26]:
df

,game_id,home_team_abbr,away_team_abbr,date,is_night_game,home_team_win,home_pitcher,away_pitcher,home_trueskill_pre,away_trueskill_pre,...,away_team_rest,home_pitcher_rest,away_pitcher_rest,team_rest_diff,pitcher_rest_diff,season,month,week_num,dow,dh_game_no
0,NYMSTL20240301D,STL,NYM,1709251200,True,True,No Pitcher,No Pitcher,25.000000,25.000000,...,30,30,30,0,0,2024,3,9,4,NaN
1,SDMIL20240301D,MIL,SD,1709251200,True,True,No Pitcher,No Pitcher,25.000000,25.000000,...,30,1,1,0,0,2024,3,9,4,NaN
2,TORNYY20240301N,NYY,TOR,1709251200,True,True,No Pitcher,No Pitcher,25.000000,25.000000,...,30,1,1,0,0,2024,3,9,4,NaN
3,PHIDET20240301D,DET,PHI,1709251200,True,True,No Pitcher,No Pitcher,25.000000,25.000000,...,30,1,1,0,0,2024,3,9,4,NaN
4,TEXSF20240301D,SF,TEX,1709251200,True,True,No Pitcher,No Pitcher,25.000000,25.000000,...,30,1,1,0,0,2024,3,9,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,SFCOL20240508N,COL,SF,1715126400,True,False,No Pitcher,No Pitcher,2.632843,22.781176,...,2,1,1,1,0,2024,5,19,2,NaN
931,ATLCWS20240627D,CWS,ATL,1719446400,True,False,No Pitcher,No Pitcher,17.769791,30.616462,...,30,30,30,0,0,2024,6,26,3,NaN
932,CLEMIN20240809D,MIN,CLE,1723161600,True,False,No Pitcher,No Pitcher,32.009877,22.801961,...,30,30,30,0,0,2024,8,32,4,NaN
933,MILCIN20240830D,CIN,MIL,1724976000,True,False,No Pitcher,No Pitcher,9.995011,22.251200,...,30,21,21,0,0,2024,8,35,4,NaN


In [27]:
batting

,ab,r,h,rbi,bb,so,pa,batting_avg,onbase_perc,slugging_perc,...,wpa_bat,leverage_index_avg,wpa_bat_pos,wpa_bat_neg,re24_bat,po,a,game_id,team,is_home_team
0,33,3,8,3,4,11,37,0.242,0.324,0.333,...,NaN,NaN,NaN,NaN,NaN,27,13,NYMSTL20240301D,STL,True
1,31,2,7,2,3,4,34,0.226,0.294,0.355,...,NaN,NaN,NaN,NaN,NaN,24,4,NYMSTL20240301D,NYM,False
2,35,7,12,7,3,5,39,0.343,0.385,0.571,...,NaN,NaN,NaN,NaN,NaN,27,9,STLHOU20240302D,HOU,True
3,33,1,6,1,4,8,37,0.182,0.270,0.303,...,NaN,NaN,NaN,NaN,NaN,24,8,STLHOU20240302D,STL,False
4,30,3,5,3,2,12,32,0.167,0.219,0.300,...,NaN,NaN,NaN,NaN,NaN,27,8,CWSLAA20240303D,LAA,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,29,2,5,2,2,11,31,0.172,0.226,0.241,...,NaN,NaN,NaN,NaN,NaN,21,3,DETNYY20240505D,DET,False
126,34,6,10,6,4,7,38,0.294,0.368,0.559,...,NaN,NaN,NaN,NaN,NaN,27,3,SFPHI20240506D,HI2,True
127,31,1,5,1,1,13,33,0.161,0.182,0.194,...,NaN,NaN,NaN,NaN,NaN,24,8,SFPHI20240506D,SFP,False
128,31,5,8,4,4,4,35,0.258,0.343,0.484,...,NaN,NaN,NaN,NaN,NaN,27,9,CWSTB20240507N,TB2,True


df

In [28]:
df.columns

Index(['game_id', 'home_team_abbr', 'away_team_abbr', 'date', 'is_night_game',
       'home_team_win', 'home_pitcher', 'away_pitcher', 'home_trueskill_pre',
       'away_trueskill_pre', 'ts_diff', 'home_team_rest', 'away_team_rest',
       'home_pitcher_rest', 'away_pitcher_rest', 'team_rest_diff',
       'pitcher_rest_diff', 'season', 'month', 'week_num', 'dow',
       'dh_game_no'],
      dtype='object')

## Add Stats

### Rolling Stats

In [29]:
# create rolling stat
b_stats = ['batting_avg','leverage_index_avg', 'onbase_perc', 'onbase_plus_slugging']
df = features.add_10RA_rolling(batting, df, b_stats, True, 'batting')

AAA
AAA
AAA
AAA
             game_id home_team_abbr away_team_abbr        date  is_night_game  \
0    NYMSTL20240301D            STL            NYM  1709251200           True   
1     SDMIL20240301D            MIL             SD  1709251200           True   
2    TORNYY20240301N            NYY            TOR  1709251200           True   
3    PHIDET20240301D            DET            PHI  1709251200           True   
4     TEXSF20240301D             SF            TEX  1709251200           True   
..               ...            ...            ...         ...            ...   
930   SFCOL20240508N            COL             SF  1715126400           True   
931  ATLCWS20240627D            CWS            ATL  1719446400           True   
932  CLEMIN20240809D            MIN            CLE  1723161600           True   
933  MILCIN20240830D            CIN            MIL  1724976000           True   
934  NYMATL20240926N            ATL            NYM  1727308800           True   

     home_t

In [30]:
df['game_id']

0      NYMSTL20240301D
1       SDMIL20240301D
2      TORNYY20240301N
3      PHIDET20240301D
4       TEXSF20240301D
            ...       
930     SFCOL20240508N
931    ATLCWS20240627D
932    CLEMIN20240809D
933    MILCIN20240830D
934    NYMATL20240926N
Name: game_id, Length: 935, dtype: object

In [31]:
pitching['SO_batters_faced'] = pitching['so'] / pitching['batters_faced']
pitching['H_batters_faced'] = pitching['h'] / pitching['batters_faced']
pitching['BB_batters_faced'] = pitching['bb'] / pitching['batters_faced']

# create rolling stat
b_stats = ['earned_run_avg','SO_batters_faced','H_batters_faced','BB_batters_faced']
df = features.add_10RA_rolling(pitching, df, b_stats, True, 'team_pitching')

AAA
AAA
AAA
AAA
             game_id home_team_abbr away_team_abbr        date  is_night_game  \
0    NYMSTL20240301D            STL            NYM  1709251200           True   
1     SDMIL20240301D            MIL             SD  1709251200           True   
2    TORNYY20240301N            NYY            TOR  1709251200           True   
3    PHIDET20240301D            DET            PHI  1709251200           True   
4     TEXSF20240301D             SF            TEX  1709251200           True   
..               ...            ...            ...         ...            ...   
930   SFCOL20240508N            COL             SF  1715126400           True   
931  ATLCWS20240627D            CWS            ATL  1719446400           True   
932  CLEMIN20240809D            MIN            CLE  1723161600           True   
933  MILCIN20240830D            CIN            MIL  1724976000           True   
934  NYMATL20240926N            ATL            NYM  1727308800           True   

     home_t

In [32]:
pitchers['earned_run_avg'] = pd.to_numeric(pitchers['earned_run_avg'], errors='coerce')
pitchers['SO_batters_faced'] = pitchers['so'] / pitchers['batters_faced']
pitchers['H_batters_faced'] = pitchers['h'] / pitchers['batters_faced']
pitchers['BB_batters_faced'] = pitchers['bb'] / pitchers['batters_faced']

# create rolling stat
b_stats = ['earned_run_avg','SO_batters_faced','H_batters_faced','BB_batters_faced']
df = features.add_10RA_rolling(pitchers, df, b_stats, False, 'pitcher')

             game_id home_team_abbr away_team_abbr        date  is_night_game  \
0    NYMSTL20240301D            STL            NYM  1709251200           True   
1     SDMIL20240301D            MIL             SD  1709251200           True   
2    TORNYY20240301N            NYY            TOR  1709251200           True   
3    PHIDET20240301D            DET            PHI  1709251200           True   
4     TEXSF20240301D             SF            TEX  1709251200           True   
..               ...            ...            ...         ...            ...   
930   SFCOL20240508N            COL             SF  1715126400           True   
931  ATLCWS20240627D            CWS            ATL  1719446400           True   
932  CLEMIN20240809D            MIN            CLE  1723161600           True   
933  MILCIN20240830D            CIN            MIL  1724976000           True   
934  NYMATL20240926N            ATL            NYM  1727308800           True   

     home_team_win home_pit

### Game Stats

In [18]:
print(games.columns)
print(df.columns)

Index(['away_team_abbr', 'home_team_abbr', 'date', 'start_time', 'venue',
       'away_team_errors', 'home_team_errors', 'away_team_hits',
       'home_team_hits', 'away_team_runs', 'home_team_runs', 'game_id',
       'is_night_game', 'is_grass', 'spread'],
      dtype='object')
Index(['name', 'ip', 'h', 'r', 'er', 'bb', 'so', 'hr', 'earned_run_avg',
       'batters_faced', 'pitches', 'strikes_total', 'strikes_contact',
       'strikes_swinging', 'strikes_looking', 'inplay_gb_total',
       'inplay_fb_total', 'inplay_ld', 'inplay_unk', 'game_score',
       'inherited_runners', 'inherited_score', 'wpa_def', 'leverage_index_avg',
       're24_def', 'game_id', 'is_home_team', 'is_starting_pitcher', 'team',
       'SO_batters_faced', 'H_batters_faced', 'BB_batters_faced',
       'earned_run_avg_10RA', 'SO_batters_faced_10RA', 'H_batters_faced_10RA',
       'BB_batters_faced_10RA'],
      dtype='object')


In [16]:
# Assuming 'game_id' is the common key and 'date' is the column you need from 'games'
df = df.merge(games[['game_id', 'home_team_abbr']], on='game_id', how='left')

In [33]:
import pandas as pd

# Example DataFrame creation
# df = pd.DataFrame({
#     'game_id': ['1', '2', '3'],
#     'date': ['2024-03-01', '2023-10-01', '2023-05-01']
# })

# Check if 'date' column is datetime type, if not convert it
if df['date'].dtype != '<M8[ns]':  # '<M8[ns]' is numpy notation for datetime64
    df['date'] = pd.to_datetime(df['date'])

# Extract the year from the date
df['season'] = df['date'].dt.year

In [34]:
df = features.game_stats(games,df)
df.shape

(935, 81)

### Season Stats

In [35]:
batting_stats = ['ab', 'bb', 'h', 'pa', 'po', 'r', 'rbi', 'so', 'batting_avg',
             'leverage_index_avg', 'onbase_perc', 'onbase_plus_slugging', 'pitches', 
             're24_bat', 'slugging_perc', 'strikes_total', 'wpa_bat', 'wpa_bat_neg', 
             'wpa_bat_pos']
df = features.add_season_rolling(batting, df, batting_stats, True,'batting')
df.shape

(935, 214)

In [36]:
pitching_stats = ['bb', 'er', 'h', 'hr', 'ip', 'r', 'so', 'batters_faced',
               'earned_run_avg', 'game_score', 'inherited_runners',
               'inherited_score', 'inplay_fb_total', 'inplay_gb_total', 'inplay_ld',
               'inplay_unk', 'leverage_index_avg', 'pitches', 're24_def',
               'strikes_contact', 'strikes_looking', 'strikes_swinging',
               'strikes_total', 'wpa_def','SO_batters_faced','H_batters_faced',
                'BB_batters_faced']
df = features.add_season_rolling(pitching, df, pitching_stats, True,'team_pitching')
df.shape

(935, 403)

In [37]:
if 'away_pitcher' not in df.columns:
    df['away_pitcher'] = 'Unknown'
else:
    df['away_pitcher'].fillna('Unknown', inplace=True)

In [38]:
df = features.add_season_rolling(pitchers, df, pitching_stats, False,'pitcher')
df.shape

(935, 592)

In [39]:
df.to_csv('../data/df_before_cleanup.csv', index=False)

# Cleanup

In [40]:
#fin na's
df = util.fix_na(df, False)

In [41]:
df.shape

(935, 592)

In [42]:
df.to_csv('../data/df_w_features.csv', index=False)

In [43]:
import pandas as pd
import numpy as np

def create_pitchers_with_starting():
    # Load the original CSV file
    pitchers = pd.read_csv('../data/pitchers.csv')
    
    # Assign random True/False to 'is_starting_pitcher'
    # np.random.choice takes a list of values and a size, generating a random selection for each row in DataFrame
    pitchers['is_starting_pitcher'] = np.random.choice([True, False], size=len(pitchers))

    # Save the modified DataFrame to a new CSV file
    pitchers.to_csv('../data/pitchers_with_starting.csv', index=False)

# Run the function to create the new CSV file
create_pitchers_with_starting()

In [44]:
import pandas as pd
import numpy as np

def create_batting_with_team():
    batting = pd.read_csv('../data/batting.csv')
    
    batting['team'] = np.where(
        batting['home_away'] == 'away', 
        batting['game_id'].str[:3],   # Extracts the first three characters if away
        batting['game_id'].str[3:6]   # Extracts the next three characters if home
    )

    # Save the modified DataFrame to a new CSV file
    batting.to_csv('../data/batting_with_team.csv', index=False)

# Run the function to create the new CSV file
create_batting_with_team()

In [ ]:
austin = True